In [1]:
!pip install openai==0.28

Defaulting to user installation because normal site-packages is not writeable


Loading the Gemstone Database


In [ ]:
import pandas as pd
gemstone_df = pd.read_csv(r'C:\Users\parul\OneDrive\Documents\GenAI\ShopAssist AI\Project_BYOP\GemstonesRecommendation.csv')
gemstone_df.head()

In [10]:
import os,json,ast
import openai
import pandas as pd

In [11]:
openai.api_key = open("api_key.txt", "r").read().strip()


Implementing Intent Clarity and Intent Confirmation Layers

In [12]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"
    example_user_req = {'Rashi': 'Aries','Gender': 'Male','Age': '25','Color':'Green','Lifestyle':'Active','Budget': '6000'}

    system_message = f"""

    You are an expert sales person and your goal is to find the best gemstone for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('Rashi','Gender','Age','Color','Lifestyle',Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this {{'Rashi': 'values','Gender': 'values','Age': 'values','Color':'values','Lifestyle':'values','Budget': 'values'}}
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    The value for 'budget' should be a numerical value extracted from the user's response.
    The values currently in the dictionary are only representative values.

    {delimiter}Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised.
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 2000 $. If the user says less than that, please mention that there are no gemstones in that range.
    - Do not randomly assign values to any of the keys. The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    {delimiter} Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the gemstone is unclear. Ask another question to comprehend their needs.
    You are trying to fill the values of all the keys ('Rashi','Gender','Age','Color','Lifestyle',Budget') in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    Answer "Yes" or "No" to indicate if you understand the requirements and have updated the values for the relevant keys. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n{delimiter}

    {delimiter}Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys. Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    Answer "Yes" or "No" to indicate if you understood all the values for the keys and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.{delimiter}

    {delimiter}Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions. {delimiter}

    Follow the above chain of thoughts and only output the final updated python dictionary. \n


    {delimiter} Here is a sample conversation between the user and assistant:
    User: "Hi, I'm interested in getting a gemstone. Can you help me find the right one?"
    Assistant: "Of course! I'd be happy to help you find the perfect gemstone. To get started, could you please provide me with some information? For example, do you have a specific Rashi or color in mind?"
    User: " Well, I'm a Taurus, and I've always liked the color green. Any recommendations?"
    Assistant: "Could you kindly let me know your budget for the gemstone? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "I'm looking to spend between $5000 and $7000.""
    Assistant: "{example_user_req}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation

In [13]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n\n    You are an expert sales person and your goal is to find the best gemstone for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'Rashi\',\'Gender\',\'Age\',\'Color\',\'Lifestyle\',Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this {\'Rashi\': \'values\',\'Gender\': \'values\',\'Age\': \'values\',\'Color\':\'values\',\'Lifestyle\':\'values\',\'Budget\': \'values\'}\n    The values for all keys, except \'budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as stated by user.\n    The value for \'budget\' should be a numerical value extracted from the user\'s response.\n    The values currently in the dictionary are only representative values.\n\n    #

In [14]:
def get_chat_model_completions(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [15]:
debug_introduction = get_chat_model_completions(debug_conversation)
print(debug_introduction)

Assistant: "Hello! Welcome to our gemstone store. I'm here to help you find the perfect gemstone. Could you please provide me with some information about your requirements? For example, do you have a specific Rashi or color in mind?"


In [16]:
debug_user_input = "Hi, I am Ashish. I am looking for a unique gemstone."

In [17]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

Of course, Ashish! I'd be happy to help you find the perfect gemstone. To get started, could you please provide me with some information? For example, do you have a specific Rashi or color in mind?


In [18]:
def moderation_check(user_input):
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]
    if moderation_output["flagged"] == True:
        return "Flagged"
    else:
        return "Not Flagged"

In [19]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

Not Flagged


In [20]:
print(moderation_check("I want to kill you."))
print(moderation_check("I need a gemstone for smuggling."))

Flagged
Not Flagged


In [21]:
moderation_check(debug_response_assistant)

'Not Flagged'

In [24]:
def intent_confirmation_layer(response_assistant):
    delimiter = "####"
    prompt = f"""
    You are a senior Sales manager who has an eye for detail.
    You are provided an input. You need to evaluate if the input has the following keys: 'Rashi','Gender','Age','Color','Lifestyle',Budget'
    Next you need to evaluate if the keys have the the values filled correctly.
    The values for all keys, except 'budget', should be 'low', 'medium', or 'high' based on the importance as stated by user. The value for the key 'budget' needs to contain a number with currency.
    Output a string 'Yes' if the input contains the dictionary with the values correctly filled for all keys.
    Otherwise out the string 'No'.

    Here is the input: {response_assistant}
    Only output a one-word string - Yes/No.
    """


    confirmation = openai.Completion.create(
                                    model="gpt-3.5-turbo-instruct",
                                    prompt = prompt,
                                    temperature=0)


    return confirmation["choices"][0]["text"]

In [25]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
print(debug_confirmation)


No


In [31]:
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})

In [32]:
debug_response_assistant_n = f"""

Based on the information you have provided, I have updated the values in the dictionary as follows:
{{'Rashi': 'high','Gender': 'low','Age': 'high','Color':'high','Lifestyle':'low','Budget': '5000'}}

Please note that the values for 'Gender' and 'Lifestyle' is set to 'low' since they are not a priority for your task.

Is there anything else I can assist you with?
"""

In [33]:
intent_confirmation_layer(debug_response_assistant_n)

'\nYes'

In [34]:
def dictionary_present(response):
    delimiter = "####"
    user_req = {'Rashi': 'high','Gender': 'high','Age': 'high','Color':'high','Lifestyle':'low','Budget': '5000'}
    prompt = f"""You are a Sales person. You are provided an input.
        You have to check if there is a python dictionary present in the string.
        It will have the following format {user_req}.
        Your task is to just extract and return only the python dictionary from the input.
        The output should match the format as {user_req}.
        The output should contain the exact keys and values as present in the input.

        Here are some sample input output pairs for better understanding:
        {delimiter}
        input: - 'Rashi': 'high', 'Gender': 'medium', 'Age': 'high', 'Color': 'high', 'Lifestyle': 'low', 'Budget': '7000'
        output: {{'Rashi': 'high', 'Gender': 'medium', 'Age': 'high', 'Color': 'high', 'Lifestyle': 'low', 'Budget': '7000'}}

        input: {{'Rashi': 'high',   'Gender': 'medium',   'Age': 'high',   'Color': 'high',    'Lifestyle': 'low',   'Budget': '7000'}}
        output: {{'Rashi': 'high', 'Gender': 'medium', 'Age': 'high', 'Color': 'high', 'Lifestyle': 'low', 'Budget': '9000'}}

        input: Here is your user profile 'Rashi': 'high', 'Gender': 'medium', 'Age': 'high', 'Color': 'high', 'Lifestyle': 'low', 'Budget': '10000'
        output: {{'Rashi': 'high', 'Gender': 'medium', 'Age': 'high', 'Color': 'high', 'Lifestyle': 'low', 'Budget': '10000'}}
        {delimiter}

        Here is the input {response}

        """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a sales person."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["choices"][0]["message"]["content"]

response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)

Here is the input:
'Here is your user profile {"Rashi": "high", "Gender": "medium", "Age": "high", "Color": "high", "Lifestyle": "low", "Budget": "10000"}'

The extracted dictionary from the input is:
{"Rashi": "high", "Gender": "medium", "Age": "high", "Color": "high", "Lifestyle": "low", "Budget": "10000"}

Is there anything else I can assist you with?


In [37]:
debug_response_assistant_n = f"""Thank you for providing your budget. Based on your budget of 10000 $, I will consider this while recommending suitable gemstone options for you.
Here is the final recommendation for your gemstone:

- Rashi: high
- Gender: low
- Age: high
- Color: high
- Lifestyle: medium
- Budget: 10000 $

Please note that these specifications are based on your requirements for driving and a city drive with great mileage within your budget. Let me know if there's anything else I can assist you with!"""

In [38]:
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)

{'Rashi': 'high', 'Gender': 'low', 'Age': 'high', 'Color': 'high', 'Lifestyle': 'medium', 'Budget': '10000 $'}


In [60]:
debug_conversation = initialize_conversation()
debug_introduction = get_chat_model_completions(debug_conversation)
debug_user_input = "Hi, I am Ashish. I need a gemstone."
debug_moderation = moderation_check(debug_user_input)
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
debug_moderation = moderation_check(debug_response_assistant)
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
response_dict_n = dictionary_present(debug_response_assistant_n)

Implementing the Product Mapping and Information Extraction Layers

In [87]:
def product_map_layer(gemstone_desc):

  delimiter = "#####"
 
  gemstone_spec = {
    "Rashi": "(Zodiac sign compatibility)",
    "Gender": "(Male, Female, Unisex)",
    "Age": "(Suitable age range)",
    "Color": "(Variety of colors available)",
    "Lifestyle": "(Active, Artistic,Professional,Urban,Outdoor,Homebody)",
   
}

  values = {'low','medium','high'}

  prompt=f"""
  You are a Gemstone sales person whose job is to pitch the key features of gemstone as per their requirements.
  To analyze each gemstone, perform the following steps:
  Step 1: Extract the gemstone's primary features from the description {gemstone_desc}
  Step 2: Store the extracted features in {gemstone_spec} \
  Step 3: Classify each of the items in {gemstone_spec}into {values} based on the following rules: \
  
  {delimiter}
  Rashi: low: < for minimal influence on specific zodiac signs >, \\n
  medium: < for a balanced influence on various zodiac signs > , \\n
  high: < for a strong positive influence on specific zodiac signs > , \\n 

  Gender :low: < for a general appeal to both genders > , \\n
  medium:< for a balanced appeal to both genders > , \\n
  high:< for a strong appeal to a specific gender > , \\n

  Age:low: < for a wide age range > , \\n
  medium:< for a moderate age range > , \\n
  high: < for a specific age group > , \\n

  Color:low: < for a subtle or neutral color range > , \\n
  medium: < for a versatile color range suitable for various occasions > , \\n
  high: < for a vibrant and bold color range > ,\\n

  Lifestyle:low: < for a gemstone suitable for everyday wear > , \\n
  medium: < for a gemstone suitable for both casual and formal occasions > , \\n
  high:< for a gemstone ideal for special and formal occasions > , \\n

 
  {delimiter}

  {delimiter}
  Here are some input output pairs for few-shot learning:
    
  input1: "This gemstone is tailored for individuals born under the Virgo zodiac sign. It offers moderate alignment benefits and is suitable for both men and women. With a range of colors, it complements various styles and occasions. Priced at 5000 it provides an affordable option for those seeking zodiac-specific gemstones."
  output1: {{'Rashi': 'medium', 'Gender': 'medium', 'Age': 'medium', 'Color': 'medium', 'Lifestyle': 'medium', 'Budget': '5000'}}

  input2: "A versatile gemstone designed for individuals of all zodiac signs. It provides balanced energy alignment, making it suitable for both genders. Available in a variety of colors, it adapts to different styles and occasions. Priced at 7500, it offers a universal and inclusive option for gemstone enthusiasts."
  output2: {{'Rashi': 'medium', 'Gender': 'medium', 'Age': 'medium', 'Color': 'high', 'Lifestyle': 'medium', 'Budget': '7500'}}

  input3: "An exclusive gemstone crafted for those seeking precise zodiac compatibility. It offers powerful alignment benefits for specific zodiac signs and is suitable for both men and women. With a rare color range, it caters to unique preferences. Priced at 14000, it represents a premium choice for discerning gemstone connoisseurs."
  output3: {{'Rashi': 'high', 'Gender': 'high', 'Age': 'medium', 'Color': 'high', 'Lifestyle': 'high', 'Budget': '14000'}}

  
  
  
  {delimiter}

  Follow the above instructions step-by-step and output the dictionary {gemstone_spec} for the following laptop {gemstone_desc}.
  """



  response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens = 2000
    
)
  return  response["choices"][0]["text"]

In [88]:
print(gemstone_df.columns)

Index(['Rashi', 'Gemstone', 'Weight (carats)', 'Price ($)', 'Color', 'Clarity',
       'Hardness', 'Cut', 'Origin', 'Treatments', 'Age Group', 'Gender',
       'Budget Range ($)', 'Style', 'Lifestyle', 'Recommendation Score',
       'Description', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'gemstone_feature'],
      dtype='object')


In [89]:
gemstone_df['gemstone_feature'] = gemstone_df['Description'].apply(lambda x: product_map_layer(x))

In [90]:
gemstone_df['gemstone_feature']

0     \n{'Rashi': 'low', 'Gender': 'medium', 'Age': ...
1     \n  {'Rashi': 'medium', 'Gender': 'medium', 'A...
2     \n  Output: \n  {'Rashi': 'low', 'Gender': 'un...
3     \n  {'Rashi': 'low', 'Gender': 'unisex', 'Age'...
4     \nOutput: {'Rashi': 'low', 'Gender': 'medium',...
5     \n  {'Rashi': 'low', 'Gender': 'medium', 'Age'...
6     output:{'Rashi': 'low', 'Gender': 'unisex', 'A...
7     \n  {'Rashi': 'low', 'Gender': 'medium', 'Age'...
8     \n  {'Rashi': 'high', 'Gender': 'unisex', 'Age...
9     \n{'Rashi': 'high', 'Gender': 'unisex', 'Age':...
10    \n  Output: {'Rashi': 'low', 'Gender': 'unisex...
11    \n  {'Rashi': 'medium', 'Gender': 'medium', 'A...
12    \n\n{'Rashi': 'low', 'Gender': 'low', 'Age': '...
13    \n  {'Rashi': 'low', 'Gender': 'low', 'Age': '...
14    \n  {'Rashi': 'low', 'Gender': 'medium', 'Age'...
15    \nOutput: {'Rashi': 'medium', 'Gender': 'low',...
16    \n{'Rashi': 'low', 'Gender': 'low', 'Age': 'wi...
17    \n{'Rashi': 'low', 'Gender': 'low', 'Age':

In [91]:
gemstone_df.to_csv(r"C:\Users\parul\OneDrive\Documents\GenAI\ShopAssist AI\Project_BYOP\updated_gemstones.csv", index=False, header=True)


In [92]:
import ast
import re

def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

In [93]:
string = " \n \t           Output: {{'Rashi': 'medium', 'Gender': 'medium', 'Age': 'medium', 'Color': 'high', 'Lifestyle': 'medium', 'Budget': '7500'}}ad"
extracted_dict = extract_dictionary_from_string(string)
print(extracted_dict)

{'rashi': 'medium', 'gender': 'medium', 'age': 'medium', 'color': 'high', 'lifestyle': 'medium', 'budget': '7500'}


In [112]:
import pandas as pd
import re

def extract_numeric_value(value):
    numeric_values = re.findall(r'\d+\.\d+|\d+', str(value))
    return float(numeric_values[0]) if numeric_values else 0

def compare_gemstones_with_user(user_req_string):
    gemstone_df = pd.read_csv(r'C:\Users\parul\OneDrive\Documents\GenAI\ShopAssist AI\Project_BYOP\updated_gemstones.csv')
    user_requirements = extract_dictionary_from_string(user_req_string)
    budget = int(user_requirements.get('budget',0).replace(',', '').split()[0])
    filtered_gemstones = gemstone_df.copy()
    filtered_gemstones['Price ($)'] = filtered_gemstones['Price ($)'].apply(extract_numeric_value)
    filtered_gemstones = filtered_gemstones[filtered_gemstones['Price ($)'] <= budget].copy()
 
       
    mappings = {
        'low': 0,
        'medium': 1,
        'high': 2
    }

   
    filtered_gemstones['Score'] = 0

   
    for index, row in filtered_gemstones.iterrows():
        user_product_match_str = row['gemstone_feature']
        gemstone_values = extract_dictionary_from_string(user_product_match_str)
        score = 0

        for key, user_value in extract_dictionary_from_string(user_req_string).items():
            if key.lower() == 'budget':
                continue  
            gemstone_value = gemstone_values.get(key, None)
            gemstone_mapping = mappings.get(gemstone_value.lower(), -1)
            user_mapping = mappings.get(user_value.lower(), -1)
            if gemstone_mapping >= user_mapping:
                score += 1

        filtered_gemstones.loc[index, 'Score'] = score

    
    top_gemstones = filtered_gemstones.drop('gemstone_feature', axis=1)
    top_gemstones = top_gemstones.sort_values('Score', ascending=False).head(3)

    return top_gemstones.to_json(orient='records')





In [113]:
print(response_dict_n)

{'Rashi': 'high', 'Gender': 'low', 'Age': 'high', 'Color': 'high', 'Lifestyle': 'medium', 'Budget': '10000'}


In [114]:
#response_dict_n = "User's requirements string here"
top_3_gemstones = compare_gemstones_with_user(response_dict_n)
print(top_3_gemstones)

[{"Rashi":"Libra","Gemstone":"Opal","Weight (carats)":1.8,"Price ($)":3800.0,"Color":"Multicolor","Clarity":"A","Hardness":"5.5-6.5","Cut":"Cabochon","Origin":"Australia","Treatments":"None","Age Group":"25-35","Gender":"Male","Budget Range ($)":"3500-10000","Style":"Classic","Lifestyle":"Active","Recommendation Score":0.8,"Description":"The iridescent opal,with its play of colors,symbolizes creativity, inspiration,and emotional healing.","Unnamed: 17":" ","Unnamed: 18":" ","Unnamed: 19":null,"Unnamed: 20":" ","Score":3},{"Rashi":"Aries","Gemstone":"Ruby","Weight (carats)":2.5,"Price ($)":3000.0,"Color":"Red","Clarity":"VVS1","Hardness":"9","Cut":"Oval","Origin":"Myanmar","Treatments":"Heat","Age Group":"25-35","Gender":"Female","Budget Range ($)":"3000-8000","Style":"Vintage","Lifestyle":"Artistic","Recommendation Score":0.7,"Description":"The vibrant red ruby  associated with passion and strength  is believed to bring good fortune.","Unnamed: 17":null,"Unnamed: 18":null,"Unnamed: 19"

In [115]:
def recommendation_validation(gemstone_recommendation):
    data = json.loads(gemstone_recommendation)
    validated_gemstones = []

    for gemstone in data:
        if gemstone.get('Score', 0) > 2:
            validated_gemstones.append(gemstone)

    return validated_gemstones


In [116]:

validated_gemstones = recommendation_validation(top_3_gemstones)
print(validated_gemstones)


[{'Rashi': 'Libra', 'Gemstone': 'Opal', 'Weight (carats)': 1.8, 'Price ($)': 3800.0, 'Color': 'Multicolor', 'Clarity': 'A', 'Hardness': '5.5-6.5', 'Cut': 'Cabochon', 'Origin': 'Australia', 'Treatments': 'None', 'Age Group': '25-35', 'Gender': 'Male', 'Budget Range ($)': '3500-10000', 'Style': 'Classic', 'Lifestyle': 'Active', 'Recommendation Score': 0.8, 'Description': 'The iridescent opal,with its play of colors,symbolizes creativity, inspiration,and emotional healing.', 'Unnamed: 17': ' ', 'Unnamed: 18': ' ', 'Unnamed: 19': None, 'Unnamed: 20': ' ', 'Score': 3}, {'Rashi': 'Aries', 'Gemstone': 'Ruby', 'Weight (carats)': 2.5, 'Price ($)': 3000.0, 'Color': 'Red', 'Clarity': 'VVS1', 'Hardness': '9', 'Cut': 'Oval', 'Origin': 'Myanmar', 'Treatments': 'Heat', 'Age Group': '25-35', 'Gender': 'Female', 'Budget Range ($)': '3000-8000', 'Style': 'Vintage', 'Lifestyle': 'Artistic', 'Recommendation Score': 0.7, 'Description': 'The vibrant red ruby  associated with passion and strength  is believe

In [117]:
len(validated_gemstones)

3

In [118]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an expert gemstone sales person and you are tasked with the objective to \
    provide details of the gemstone from the catalogue: {products}.\
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each gemstone in the following format, in decreasing order of price of gemstones:
    1. <Gemstone Name>: <Key features of the gemstone>, <Price in $>
    2. <Gemstone Name>: <Key features of the gemstone>, <Price in $>

    """
    conversation = [{"role": "system", "content": system_message }]
    return conversation

In [119]:
debug_conversation_reco = initialize_conv_reco(top_3_gemstones)

In [120]:
debug_recommendation = get_chat_model_completions(debug_conversation_reco)
print(debug_recommendation + '\n')

Gemstone Catalogue:

1. Opal: The iridescent opal, with its play of colors, symbolizes creativity, inspiration, and emotional healing. It has a weight of 1.8 carats and is priced at $3800. It is multicolored and has a clarity of A. The opal has a hardness of 5.5-6.5 and is cut in a cabochon style. It originates from Australia and has no treatments. It is recommended for individuals in the age group of 25-35, specifically males with a budget range of $3500-10000. The opal is a classic gemstone suitable for an active lifestyle. The recommendation score for this gemstone is 0.8.

2. Ruby: The vibrant red ruby is associated with passion and strength and is believed to bring good fortune. It has a weight of 2.5 carats and is priced at $3000. The ruby has a color of red and a clarity of VVS1. It has a hardness of 9 and is cut in an oval shape. The ruby originates from Myanmar and has undergone heat treatment. It is recommended for individuals in the age group of 25-35, specifically females w

In [121]:
debug_conversation_reco.append({"role": "user", "content": "This is my user profile" + response_dict_n})
debug_conversation_reco.append({"role": "assistant", "content": debug_recommendation})

In [124]:
debug_user_input = "Which is ideal for good fortune?"

In [125]:
debug_conversation_reco.append({"role": "user", "content": debug_user_input})
debug_response_asst_reco = get_chat_model_completions(debug_conversation_reco)
print('\n' + debug_response_asst_reco + '\n')


The gemstone ideal for good fortune is Ruby. It is believed to bring good fortune and is associated with passion and strength.



Dialogue Management System

In [126]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()
    introduction = get_chat_model_completions(conversation)
    print(introduction + '\n')
    top_3_cars = None
    user_input = ''

    while(user_input != "exit"):
        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if top_3_gemstones is None:
            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_model_completions(conversation)

            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            confirmation = intent_confirmation_layer(response_assistant)

            moderation = moderation_check(confirmation)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            if "No" in confirmation:
                conversation.append({"role": "assistant", "content": response_assistant})
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
            else:
                print("\n" + response_assistant + "\n")
                print('\n' + confirmation + '\n')
                response = dictionary_present(response_assistant)

                moderation = moderation_check(response)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                print('\n' + response + '\n')
                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_3_gemstones = compare_gemstones_with_user(response)

                validated_reco = recommendation_validation(top_3_gemstones)

                if len(validated_reco) == 0:
                    print("Sorry, we do not have gemstones that match your requirements. Connecting you to a human expert.")
                    break

                conversation_reco = initialize_conv_reco(validated_reco)
                recommendation = get_chat_model_completions(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "user", "content": "This is my user profile" + response})

                conversation_reco.append({"role": "assistant", "content": recommendation})

                print(recommendation + '\n')

        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_model_completions(conversation_reco)

            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})

In [138]:
gemstone_special_conversation = [
    {"role": "assistant", "content": "Greetings! I'm a gemstone expert, and I'm here to help you find the perfect gemstone for your preferences. Could you share a bit about yourself and what you're looking for in a gemstone?"},
    {"role": "user", "content": "Hello! I'm a person who loves unique gemstones. I'm specifically interested in a gemstone that reflects my zodiac sign (Libra) and can be a statement piece for special occasions."},
    {"role": "assistant", "content": "Fantastic! Unique gemstones that align with your zodiac sign can indeed be exquisite statement pieces. To narrow down the options, are there any particular colors or styles you find appealing?"},
    {"role": "user", "content": "I'm drawn to deep blue and purple hues, and I prefer elegant and classic styles."},
    {"role": "assistant", "content": "Great choices! Deep blue and purple gemstones exude elegance. Now, considering your preferences, let's talk about your budget. What's your budget range for the gemstone you have in mind?"},
    {"role": "user", "content": "I'm willing to spend around 4000 $."}
]


special_conversation_initialize = initialize_conversation()
special_conversation_initialize.extend(gemstone_special_conversation)
special_conversation_assistant_output = get_chat_model_completions(special_conversation_initialize)
special_response_dict = dictionary_present(special_conversation_assistant_output)
special_dict = extract_dictionary_from_string(special_response_dict)
print(special_dict)

special_tagged_dict={'rashi': 'medium', 'gender': 'medium', 'age': 'medium', 'color': 'high', 'lifestyle': 'medium', 'budget': '5500'}
print(special_tagged_dict)

{'rashi': 'libra', 'gender': 'values', 'age': 'values', 'color': 'high', 'lifestyle': 'values', 'budget': 4000}
{'rashi': 'medium', 'gender': 'medium', 'age': 'medium', 'color': 'high', 'lifestyle': 'medium', 'budget': '5500'}


In [153]:
gemstone_academic_conversation = [
    {"role": "assistant", "content": "Hello! I'm a gemstone expert, and I'm here to help you find the perfect gemstone based on your preferences. Could you tell me a bit about yourself and what you're looking for in a gemstone?"},
    {"role": "user", "content": "I'm an academic researcher, and I'm interested in buying a gemstone for personal and spiritual reasons."},
    {"role": "assistant", "content": "That's wonderful! Gemstones can have unique properties and meanings. Could you share more about the specific qualities you are seeking in a gemstone? For example, are you looking for a gemstone associated with a particular zodiac sign, or do you have a preference for certain colors or styles?"},
    {"role": "user", "content": "I'm particularly interested in a gemstone that aligns with my zodiac sign (Taurus) and has calming energies."},
    {"role": "assistant", "content": "Great choice! Aligning with your zodiac sign and seeking calming energies are important criteria. Libra is associated with balance and harmony. Now, let's talk about colors. Are there specific colors that resonate with you or that you find calming?"},
    {"role": "user", "content": "I find shades of blue and green very calming."},
    {"role": "assistant", "content": "Excellent choice! Blue and green gemstones are often associated with tranquility and balance. Now, considering your preferences, would you like a gemstone with a specific setting or design, or are you open to various styles?"},
    {"role": "user", "content": "I prefer a simple and elegant design."},
    {"role": "assistant", "content": "Got it! A simple and elegant design is a timeless choice. Now, let's discuss your budget for the gemstone. What's your maximum budget in INR?"},
    {"role": "user", "content":"I'm willing to spend around 7000 $"}
]



academic_conversation_initialize = initialize_conversation()
academic_conversation_initialize.extend(gemstone_academic_conversation)
academic_conversation_assistant_output = get_chat_model_completions(academic_conversation_initialize)
academic_response_dict = dictionary_present(academic_conversation_assistant_output)
print(academic_response_dict)
academic_dict = extract_dictionary_from_string(academic_response_dict)
print(academic_dict)
academic_tagged_dict = {'rashi': 'medium', 'gender': 'medium', 'age': 'medium', 'color': 'high', 'lifestyle': 'medium', 'budget': '9500'}
print(academic_tagged_dict)

{'rashi': 'high', 'gender': 'high', 'age': 'high', 'color': 'high', 'lifestyle': 'low', 'budget': '5000'}
{'rashi': 'medium', 'gender': 'medium', 'age': 'medium', 'color': 'high', 'lifestyle': 'medium', 'budget': '9500'}


In [145]:

gemstone_astro_conversation = [
    {"role": "assistant", "content": "Hello! I'm a gemstone expert here to help you find the perfect gemstone. Can you tell me a bit about yourself and what you're looking for in a gemstone?"},
    {"role": "user", "content": "I'm interested in getting a gemstone for its astrological benefits. I'm a Sagittarius, and I want something that brings positive energy."},
    {"role": "assistant", "content": "Great choice! Astrological considerations can add a meaningful touch to your gemstone selection. Are there specific colors or gemstone types you're drawn to, or do you have any preferences in terms of style?"},
    {"role": "user", "content": "I like the idea of a blue gemstone, and I prefer something with a classic and elegant design."},
    {"role": "assistant", "content": "Got it! Blue gemstones can indeed be stunning, and a classic and elegant design adds to its charm. Now, let's talk about your budget. What's your budget range for the gemstone you're considering?"},
    {"role": "user", "content": "I'm willing to spend around 6000 $."}
]

astro_conversation_initialize = initialize_conversation()
astro_conversation_initialize.extend(gemstone_astro_conversation)
astro_conversation_assistant_output = get_chat_model_completions(astro_conversation_initialize)
astro_response_dict = dictionary_present(astro_conversation_assistant_output)
astro_dict = extract_dictionary_from_string(astro_response_dict)
print(astro_dict)

astro_tagged_dict={'rashi': 'high', 'gender': 'medium', 'age': 'medium', 'color': 'high', 'lifestyle': 'medium', 'budget': '5500'}
print(astro_tagged_dict)

{'rashi': 'sagittarius', 'gender': 'values', 'age': 'values', 'color': 'blue', 'lifestyle': 'values', 'budget': 6000}
{'rashi': 'high', 'gender': 'medium', 'age': 'medium', 'color': 'high', 'lifestyle': 'medium', 'budget': '5500'}


In [ ]:
gemstone_good_fortune_conversation = [
    {"role": "assistant", "content": "Hello! I'm delighted to assist you in discovering a gemstone that brings good fortune. Could you share a bit about yourself and your preferences in gemstones?"},
    {"role": "user", "content": "I'm seeking a gemstone for its positive energies and associations with good fortune."},
    {"role": "assistant", "content": "That's wonderful! Gemstones have unique properties, and finding one for positive energies is a great choice. To tailor the recommendations, could you specify if you have a zodiac sign preference or any specific colors that resonate with you?"},
    {"role": "user", "content": "I'm a Aries, and I'm drawn to Red colors."},
    {"role": "assistant", "content": "Fantastic! Aries are often associated with vitality and warmth. Red colors can enhance those energies.Now, may I inquire about your age?"},
    {"role": "user", "content": "I am 30 years old."},
    {"role": "assistant", "content": "To further personalize your gemstone recommendation, may I ask about your gender?"},
    {"role": "user", "content": "I identify as Female"},
    {"role": "assistant", "content":"Now, let's discuss your budget for the gemstone. What's your maximum budget in USD?"},
    {"role": "user", "content": "I'm willing to invest around $10,000."},
]

good_fortune_conversation_initialize = initialize_conversation()
good_fortune_conversation_initialize.extend(gemstone_good_fortune_conversation)
good_fortune_conversation_assistant_output = get_chat_model_completions(good_fortune_conversation_initialize)
good_fortune_response_dict = dictionary_present(good_fortune_conversation_assistant_output)
print(good_fortune_response_dict)
good_fortune_dict = extract_dictionary_from_string(good_fortune_response_dict)
print(good_fortune_dict)

good_fortune_tagged_dict={'rashi': 'high', 'gender': 'medium', 'age': 'medium', 'color': 'high', 'lifestyle': 'medium', 'budget': '7500'}
print(good_fortune_tagged_dict)

In [ ]:
def evaluate_model_response(tagged_dict, model_dict):
  score = 0
  mappings = {
      'low': 0,
      'medium': 1,
      'high': 2
  }

  for key in tagged_dict.keys():
    if key.lower() == 'budget':
      continue
    tagged_value = tagged_dict[key]
    model_value = model_dict[key]
    tagged_mapping = mappings.get(tagged_value, -1)
    model_mapping = mappings.get(model_value, -1)

    if model_mapping >= tagged_mapping:
        score += 1

  return score

In [171]:
special_score = evaluate_model_response(special_tagged_dict,special_dict)
print(special_score)
academic_score = evaluate_model_response(academic_tagged_dict,academic_dict)
print(academic_score)
astro_score = evaluate_model_response(astro_tagged_dict,astro_dict)
print(astro_score)
good_fortune_score = evaluate_model_response(good_fortune_tagged_dict,good_fortune_dict)
print(good_fortune_score)

1
4
0
1
